In [2]:
# ran it with conda env bug_env
# Now the imports
import os
import sys
#from google.colab import auth, drive
import torch
from torch.utils.data import DataLoader
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from transformers import BertConfig, BertForPreTraining, BertLMHeadModel, BertTokenizer, logging
from transformers.data.datasets.language_modeling import TextDatasetForNextSentencePrediction
from transformers import pipeline, Trainer, TrainingArguments
from transformers import AutoConfig, AutoModel
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
# from nlp import load_dataset
# from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import seqeval
import json
import math
import logging
from datetime import datetime
import pytz     # for time zone
import gzip
import csv
import pickle
import re
import copy
# set the pathnames and authenticate for my google bucket
#project_id = 'serene-mender-286105'
#!gcloud config set project {project_id}
BUCKET_NAME = 'clinical_bert_bucket'


/home/leab/anaconda3/envs/bug_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
PST = pytz.timezone('US/Pacific')
!mkdir "pytorch_finetuned_model"
!rm -rf "pytorch_finetuned_model/*"
!mkdir "pytorch_finetuned_log"
!rm -rf "pytorch_finetuned_log/*"

#instantiate the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

#instantiate the model
print("start loading model=",datetime.now(PST))
# model = BertLMHeadModel.from_pretrained("bert-base-uncased")
model = BertForPreTraining.from_pretrained("bert-base-uncased")

# define the arguments for the trainer
training_args = TrainingArguments(
    output_dir='pytorch_finetuned_model',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training (try 16 if needed)
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='pytorch_finetuned_log',     # directory for storing logs
    do_train=True,
    evaluation_strategy="steps",
    eval_steps=2
)

# prepare the training and validation data files
with open("sent_train.txt","w") as f_out:
  f_out.write("This is the first training sentence.\n")
  f_out.write("This is the second training sentence.\n")
  f_out.write("\n This is the 3 training sentence.\n")
  f_out.write("This is the 4 training sentence.\n")
  f_out.write("\nThis is the 5 training sentence.\n")
  f_out.write("This is the 6 training sentence.\n")
with open("sent_eval.txt","w") as f_out:
  f_out.write("This is the first eval sentence.\n")
  f_out.write("This is the second eval sentence.\n")
  f_out.write("\nThis is the 3 eval sentence.\n")
  f_out.write("This is the 4 eval sentence.\n")

# prepare the train_dataset
print("start building train_dataset=",datetime.now(PST))
train_dataset = TextDatasetForNextSentencePrediction(
    tokenizer=tokenizer,
    file_path="sent_train.txt",
    block_size=128
)

print("start building eval_dataset=",datetime.now(PST))
eval_dataset = TextDatasetForNextSentencePrediction(
    tokenizer=tokenizer,
    file_path="sent_eval.txt",
    block_size=128
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Instantiate the trainer
print("start building trainer=",datetime.now(PST))
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=eval_dataset            # evaluation dataset
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

import evaluate
metric = evaluate.load("accuracy", )


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # preds have the same shape as the labels, after the argmax(-1) has been calculated
    # by preprocess_logits_for_metrics
    labels = labels.reshape(-1)
    preds = preds.reshape(-1)
    mask = labels != -100
    labels = labels[mask]
    preds = preds[mask]
    return metric.compute(predictions=preds, references=labels)

print("finished=",datetime.now(PST))

start loading model= 2024-04-22 05:05:17.917044-07:00


/home/leab/anaconda3/envs/bug_env/lib/python3.9/site-packages/transformers/training_args.py:1449: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/leab/anaconda3/envs/bug_env/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:360: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


start building train_dataset= 2024-04-22 05:05:38.864242-07:00
start building eval_dataset= 2024-04-22 05:05:38.869612-07:00
start building trainer= 2024-04-22 05:05:38.872436-07:00


Using the latest cached version of the module from /home/leab/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Sun Mar 31 12:42:24 2024) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


finished= 2024-04-22 05:08:25.508310-07:00


In [5]:
# now do training
trainer.train()

Step,Training Loss,Validation Loss
2,No log,9.265621


TrainOutput(global_step=3, training_loss=3.486131032307943, metrics={'train_runtime': 0.6914, 'train_samples_per_second': 17.355, 'train_steps_per_second': 4.339, 'total_flos': 149075907840.0, 'train_loss': 3.486131032307943, 'epoch': 3.0})